In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [7]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [8]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [14]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [15]:
all_shop_df['납품요구변경차수'] = all_shop_df['납품요구변경차수'].astype(int)
all_shop_df = all_shop_df.loc[all_shop_df.groupby(['납품요구번호', '물품순번'])['납품요구변경차수'].idxmax()]
all_shop_df = all_shop_df.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [17]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '품목', '수요기관지역명', '납품요구지청명','업체기업구분명','우수제품여부']]

In [18]:
# 컬럼명을 한글로 변경
column_mapping = {
    '납품요구접수일자' : 'dlvrReqRcptDate',
    '품목' : 'prdctIdntNoNm',
    '단가' : 'prdctUprc',
    '단위' : 'prdctUnit',
    '수량' : 'prdctQty',
    '금액' : 'prdctAmt',
    '우수제품여부' : 'exclcProdctYn',
    '수요기관명' : 'dminsttNm',
    '수요기관지역명' : 'dminsttRgnNm',
    '업체명' : 'corpNm',
    '납품요구건명' : 'dlvrReqNm',
    '업체기업구분명' : 'corpEntrprsDivNmNm',
    '납품요구지청명' : 'brnofceNm'
}

all_shop_df_fin.rename(columns=column_mapping, inplace=True)

In [20]:
all_shop_df_fin

,dlvrReqRcptDate,dminsttNm,dlvrReqNm,corpNm,prdctUprc,prdctUnit,prdctQty,prdctAmt,prdctIdntNoNm,dminsttRgnNm,brnofceNm,corpEntrprsDivNmNm,exclcProdctYn
0,2024-07-22,경상남도 밀양시,무안공설운동장(풋살장) 정비공사-인조잔디,케이앤비준우(주),59800.0,㎡,0.0,0,"인조잔디, 케이앤비준우, KBEE55Z, t55mm, 충전재, 이중",경상남도 밀양시,경남지방조달청,중소기업,Y
1,2024-07-22,서울특별시교육청 서울특별시동작관악교육지원청,서울공업고등학교 운동장 환경개선공사 관급자재(인조잔디 외 6건) 구매,주식회사 그린에셀,79500.0,㎡,6860.0,545370000,"인조잔디, 그린에셀, GE55S-P, t55mm, 충전재/충격흡수패드",서울특별시 동작구,서울지방조달청,중소기업,Y
2,2024-07-22,경상북도 의성군,안계고테니스장보수공사 관급자재_인조잔디,주식회사 에버그린필드,30800.0,㎡,1224.0,37699200,"인조잔디, 에버그린필드, EGF-XM20, t20mm",경상북도 의성군,대구지방조달청,중소기업,N
3,2024-07-22,경상남도창원교육청 창원상남초등학교,2024.창원상남초 운동장 인조잔디 구매,주식회사 네오필드,61500.0,㎡,3315.0,203872500,"인조잔디, 네오필드, NEO55, t55mm, 충전재",경상남도 창원시 성산구,경남지방조달청,중소기업,Y
4,2024-07-22,강원특별자치도교육청 강원특별자치도춘천교육지원청,부안초 인조잔디운동장 조성공사 관급자재(인조잔디) 구입,(주)대건씨앤엘,62000.0,㎡,1840.0,114080000,"인조잔디, 대건씨앤엘, Wellgreen 55, t55mm, 충전재",강원도 춘천시,강원지방조달청,중소기업,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14477,2008-11-03,경기도교육청 경기도화성오산교육지원청 성호초등학교,성호초등학교 인조잔디 구매,주식회사 효성,55370.0,㎡,3788.0,209741560,"인조잔디, 효성, EG-KM60T, W3700(4000)×60mm, SEBS사출칩,...",경기도 오산시,인천지방조달청,대기업,None
14478,2008-11-03,경기도교육청 경기도화성오산교육지원청 양감초등학교,양감초 인조잔디 구매,주식회사 효성,61838.0,㎡,3875.0,239622250,"인조잔디, 효성, (TR)EG-M60T, W4000×60mm, SEBS 사출칩, 효...",경기도 화성시,인천지방조달청,대기업,None
14479,2008-11-03,강원특별자치도 홍천군,잿골 체육공원 풋살경기장 조성 인조잔디,코오롱글로텍 주식회사,44500.0,㎡,778.0,34621000,"인조잔디, 코오롱글로텍, TM5000, W4000×55mm, GR인증SBR코팅칩/컬러",강원특별자치도 홍천군,강원지방조달청,대기업,None
14480,2008-11-03,충청북도교육청 충청북도충주교육지원청,교현초 인조잔디 구매,주식회사 효성,52920.0,㎡,4354.0,230413680,"인조잔디, 효성, (TR)EG-F55T, W4000×55mm, SEBS사출칩, 효성...",충청북도 충주시,충북지방조달청,대기업,None


In [21]:
# 빅쿼리 적재
save_dataframe_to_bigquery(all_shop_df_fin,'g2b','shop_detail_df_view',bigquerykey_path)

Data inserted into table shop_detail_df_view successfully.
